In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Some Librarie and Dependencies

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [4]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Encode and get score

In [15]:
tokens = tokenizer.encode("Ummm, Ok", return_tensors="pt")

In [16]:
tokens[0]

tensor([  101, 53655, 10150,   117, 13563,   102])

In [17]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-0.7727, -0.1141,  0.6923,  0.2864, -0.1411]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [18]:
result.logits

tensor([[-0.7727, -0.1141,  0.6923,  0.2864, -0.1411]],
       grad_fn=<AddmmBackward0>)

In [19]:
int(torch.argmax(result.logits))+1

3

# Get review from yelp website

In [45]:
r = requests.get('https://www.yelp.com/biz/uncle-boys-san-francisco?osq=Burgers')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [46]:
reviews[0]

"A great no-frills burger spot in the Richmond!We were in the mood for burgers for lunch, but didn't feel like a full sit-down meal so were happy to find this great spot that was exactly what we were looking for. There is only a small counter for 2 to sit inside and a parklet with some outdoor seating so I would say this is more of a take-out place.While it may be a pretty small menu, they make everything pretty tasty! The fries had a really good house seasoning that had a great flavor and would definitely order those again or may try the onion rings instead.For burgers, I had the El Jefe and my wife had the O.G. Just like the restaurant itself, they weren't the fanciest, but they tasted great and were very filling! We might have to try the Chicken Sandwiches next time, but if we ever need a solid burger, this is a great option to come back to!"

# Put reviews and their score into a dataframe

In [31]:
import pandas as pd

In [47]:
df = pd.DataFrame(np.array(reviews), columns=["review"])
df

,review
0,A great no-frills burger spot in the Richmond!...
1,This gem of a restaurant is located in a charm...
2,"One of, if not, the only place open for a late..."
3,I went back some months later after being told...
4,Just got home and found that my Cowboy Burger ...
5,"I got the O.G Wagyu Burger, fries, and Cowboy ..."
6,They literally lure me back with their monthly...
7,Today i splurged and tried their lumpia. That...
8,"Yes, very juicy hamburger. Onion rings please..."
9,"While on vacation in San Francisco, my family ..."


In [48]:
def sent_score(review):
    tokens = tokenizer.encode(review, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [49]:
df["Score"] = df["review"].apply(lambda x: sent_score(x[:512]))

In [50]:
mapp = {1: "Terrible", 2:"Poor", 3:"Average", 4:"Good", 5:"Excellent"}

In [51]:
df["Score"] = df["Score"].map(mapp)

In [52]:
df

,review,Score
0,A great no-frills burger spot in the Richmond!...,Good
1,This gem of a restaurant is located in a charm...,Good
2,"One of, if not, the only place open for a late...",Good
3,I went back some months later after being told...,Poor
4,Just got home and found that my Cowboy Burger ...,Average
5,"I got the O.G Wagyu Burger, fries, and Cowboy ...",Excellent
6,They literally lure me back with their monthly...,Good
7,Today i splurged and tried their lumpia. That...,Poor
8,"Yes, very juicy hamburger. Onion rings please...",Average
9,"While on vacation in San Francisco, my family ...",Excellent


In [53]:
df["Score"].value_counts()

Score
Good         5
Excellent    4
Poor         2
Average      2
Name: count, dtype: int64